In [1]:
from HeaderP import *
import pynbody as pb
import pynbody.filt as filt
import os, pickle, glob


In [2]:
pd.__version__

'1.3.4'

In [3]:
def readsimdata(simNo,Rcut,Rcut_thread,zcut_thread,disk=True):
    if disk:
        data=pd.read_pickle('./g'+simNo+'_z_0_decomp.pkl')
    else:
        data=pd.read_pickle('./g'+simNo+'_z_0.pkl')
    data['R']=np.sqrt((data['x'])**2+data['y']**2)
    
    if Rcut:
        data=data.loc[data['Rbir']<Rcut_thread].reset_index(drop=True)
        data=data.loc[abs(data['zbir'])<zcut_thread].reset_index(drop=True)
    data['dR']=data['R']-data['Rbir']
    data['dz']=abs(data['z'])-abs(data['zbir'])
    if disk:
        data = data.loc[(data['decomp']==1)|(data['decomp']==4)].reset_index()
    return data


import matplotlib
from scipy import interpolate
from scipy.interpolate import interp1d
def getcolor(vrange,cm=plt.cm.get_cmap('viridis')):
    cmap = cm
    norm = matplotlib.colors.Normalize(vmin=min(vrange[:-1]), vmax=max(vrange[:-1]))
    return [cmap(norm(i)) for i in vrange]

In [4]:
import os 
sims = os.listdir('./')
sims = [i for i in sims if 'x' in i]
sims = [i for i in sims if '.txt' not in i]
sims = [i for i in sims if '.pkl' not in i]
simsnames = [i.split('_')[0][1:] for i in sims]


In [ ]:
from os.path import exists

cols = ['iord','r','z','x','y']
cmap = plt.cm.get_cmap('plasma',20)

Rbirth_switch = True

for i in range(len(sims)):
    if sims[i]=='g1.12e12_3x9':
        continue
    print(sims[i])
    simname = sims[i]
    
    allsnaps = glob.glob(sims[i]+'/*.?????')
    
    agepickle = readsimdata(simsnames[i],True, 15, 5)
    
    allsnaps_time = [int(i.split('.')[-1]) for i in allsnaps if (('param' not in i) and ('draco' not in i) and ('dat_' not in i) and ('79e12' not in i.split('.')[-1]))]
    snaplookup = {int(allsnaps[i].split('.')[-1]):allsnaps[i] for i in range(len(allsnaps)) if (('param' not in allsnaps[i]) and ('draco' not in allsnaps[i]) and ('dat_' not in allsnaps[i]) and ('79e12' not in allsnaps[i].split('.')[-1]))}

    sortage = sorted(allsnaps_time)
    redshifts = np.zeros(len(sortage))
    allages_rs = np.zeros(len(redshifts))
    for j in trange(len(sortage)):
        s = pb.load(snaplookup[sortage[j]])
        h = s.properties['h']
        ages = float(str(s.properties['time']).split(' ')[0])
        allages_rs[j] = ages
    count = 0
            
    for j in trange(len(allages_rs)):
        #print('reading ',snaplookup[sortage[j]])
        if exists(snaplookup[sortage[j]]+'.pkl'):
            this_snap = pd.read_pickle(snaplookup[sortage[j]]+'.pkl')
            if len(this_snap)==0:
                print('no stars in this snap!')
                continue
        else:
            s = pb.load(snaplookup[sortage[j]])
            h=s.halos()
            try:
                pb.analysis.angmom.faceon(h[1])
            except:
                print('no halo!')
                continue
            s.physical_units()
            sim = h[1]

            stars = sim.s
            if len(stars)==0:
                print('no stars!')
                continue
            
            try:
                stars['iord']
            except:
                print('no IDs!')
                continue
            this_snap = pd.DataFrame({k:stars[k] for k in cols})
            this_snap = pd.merge(this_snap, agepickle[['iord', 'age', 'feh', 'Rbir','zbir']], left_on='iord', right_on='iord', how='inner')
            this_snap.to_pickle(snaplookup[sortage[j]]+'.pkl')
        
        plt.figure(figsize=(10,8))
        plt.title(simsnames[i]+'; Cosmic Time = %.3f'%(allages_rs[j])+' Gyr', fontsize=25)
        if Rbirth_switch:
            plt.hexbin(this_snap['age'], this_snap['Rbir'], C = this_snap['r']-this_snap['Rbir'],
                   extent = (0,14,0,15), gridsize=40, vmin=0, vmax=4, cmap=cmap, reduce_C_function = np.std, mincnt = 20)
        else:
            plt.hexbin(this_snap['age'], this_snap['r'], C = this_snap['r']-this_snap['Rbir'],
                   extent = (0,14,0,15), gridsize=40, vmin=0, vmax=4, cmap=cmap, reduce_C_function = np.std, mincnt = 20)
        plt.xlabel('Age [Gyr]')
        if Rbirth_switch:
            plt.ylabel('$R_{bir}$')
        else:
            plt.ylabel('Current Radius')
        cb = plt.colorbar()
        cb.set_label('$σ_{R-R_{bir}}$ [kpc]')
        plt.tight_layout()
        if Rbirth_switch:
            plt.savefig('RM/'+simsnames[i]+'/'+str(count)+'_Rbir.png')
        else:
            plt.savefig('RM/'+simsnames[i]+'/'+str(count)+'.png')
        count = count+1
        plt.close()

g6.96e11_3x9


  1%|          | 1/128 [00:00<00:40,  3.13it/s]

no halo!
no stars in this snap!
no stars in this snap!


 14%|█▍        | 18/128 [00:06<00:44,  2.47it/s]

no stars in this snap!
no stars in this snap!


...

  1%|          | 2/180 [00:41<1:11:54, 24.24s/it]

no stars!
no stars in this snap!


 19%|█▉        | 34/180 [02:03<51:58, 21.36s/it] 

no IDs!


 48%|████▊     | 87/180 [03:18<02:45,  1.78s/it]

In [ ]:
from os.path import exists

cols = ['iord','r','z']
cmap = plt.cm.get_cmap('plasma',20)

for i in range(len(sims)):
    if sims[i]=='g1.12e12_3x9':
        continue
    print(sims[i])
    simname = sims[i]
    
    allsnaps = glob.glob(sims[i]+'/*.?????')
    
    agepickle = readsimdata(simsnames[i],True, 15, 5)
    
    allsnaps_time = [int(i.split('.')[-1]) for i in allsnaps if (('param' not in i) and ('draco' not in i) and ('dat_' not in i) and ('79e12' not in i.split('.')[-1]))]
    snaplookup = {int(allsnaps[i].split('.')[-1]):allsnaps[i] for i in range(len(allsnaps)) if (('param' not in allsnaps[i]) and ('draco' not in allsnaps[i]) and ('dat_' not in allsnaps[i]) and ('79e12' not in allsnaps[i].split('.')[-1]))}

    sortage = sorted(allsnaps_time)
    redshifts = np.zeros(len(sortage))
    allages_rs = np.zeros(len(redshifts))
    for j in trange(len(sortage)):
        s = pb.load(snaplookup[sortage[j]])
        h = s.properties['h']
        ages = float(str(s.properties['time']).split(' ')[0])
        allages_rs[j] = ages
    count = 0
            
    for j in trange(len(allages_rs)-1):
        print('reading ',snaplookup[sortage[j]])
        if exists(snaplookup[sortage[j]]+'.pkl'):
            this_snap = pd.read_pickle(snaplookup[sortage[j]]+'.pkl')
        else:
            s = pb.load(snaplookup[sortage[j]])
            h=s.halos()
            try:
                pb.analysis.angmom.faceon(h[1])
            except:
                print('no halo!')
                continue
            s.physical_units()
            sim = h[1]

            stars = sim.s
            if len(stars)==0:
                print('no stars!')
                continue
            
            try:
                stars['iord']
            except:
                print('no IDs!')
                continue
            this_snap = pd.DataFrame({k:stars[k] for k in cols})
            this_snap = pd.merge(this_snap, agepickle[['iord', 'age', 'feh', 'Rbir','zbir']], left_on='iord', right_on='iord', how='inner')
            this_snap.to_pickle(snaplookup[sortage[j]]+'.pkl')
            
        if exists(snaplookup[sortage[j+1]]+'.pkl'):
            next_snap = pd.read_pickle(snaplookup[sortage[j+1]]+'.pkl')
        else:
            s = pb.load(snaplookup[sortage[j+1]])
            h=s.halos()
            try:
                pb.analysis.angmom.faceon(h[1])
            except:
                print('no halo!')
                continue
            s.physical_units()
            sim = h[1]

            stars = sim.s
            if len(stars)==0:
                print('no stars!')
                continue
            
            try:
                stars['iord']
            except:
                print('no IDs!')
                continue
            next_snap = pd.DataFrame({k:stars[k] for k in cols})
            next_snap = pd.merge(next_snap, agepickle[['iord', 'age', 'feh', 'Rbir','zbir']], left_on='iord', right_on='iord', how='inner')
            next_snap.to_pickle(snaplookup[sortage[j+1]]+'.pkl')
        
        plt.figure(figsize=(10,8))
        plt.title(simsnames[i]+'; Cosmic Time = %.3f'%(allages_rs[j+1])+' Gyr', fontsize=25)
        
        timediff = allages_rs[j+1]-allages_rs[j]
        
        output = plt.hexbin(this_snap['age'], this_snap['Rbir'], C = this_snap['r']-this_snap['Rbir'],
               extent = (0,14,0,15), gridsize=40, reduce_C_function = np.std, mincnt = 20)
        output_2 = plt.hexbin(next_snap['age'], next_snap['Rbir'], C = next_snap['r']-next_snap['Rbir'],
               extent = (0,14,0,15), gridsize=40, reduce_C_function = np.std, mincnt = 20)
        diffout = (output_2.get_array - output.get_array)/timediff
        plt.imshow(diffout.T, interpolation="nearest" ,aspect = 'auto',origin = 'lower', 
               extent = (0,14,0,15),cmap=cmap)
        plt.xlabel('Age [Gyr]')
        plt.ylabel('$R_{bir}$')
        
        cb = plt.colorbar()
        cb.set_label('diff($σ_{R-R_{bir}}$)/diff(\tau) [kpc/Gyr]')
        
        plt.tight_layout()
        plt.savefig('RM/'+simsnames[i]+'/'+str(count)+'_Rbir_diff.png')
        count = count+1
        plt.close()

In [ ]:

i=1
print(sims[i])
simname = sims[i]

allsnaps = glob.glob(sims[i]+'/*.?????')

agepickle = readsimdata(simsnames[i],True, 15, 5)

allsnaps_time = [int(i.split('.')[-1]) for i in allsnaps if (('param' not in i) and ('draco' not in i) and ('dat_' not in i) and ('79e12' not in i.split('.')[-1]))]
snaplookup = {int(allsnaps[i].split('.')[-1]):allsnaps[i] for i in range(len(allsnaps)) if (('param' not in allsnaps[i]) and ('draco' not in allsnaps[i]) and ('dat_' not in allsnaps[i]) and ('79e12' not in allsnaps[i].split('.')[-1]))}

sortage = sorted(allsnaps_time)
redshifts = np.zeros(len(sortage))
allages_rs = np.zeros(len(redshifts))
for j in trange(len(sortage)):
s = pb.load(snaplookup[sortage[j]])
h = s.properties['h']
ages = float(str(s.properties['time']).split(' ')[0])
allages_rs[j] = ages
count = 0

j=50
print('reading ',snaplookup[sortage[j]])
if exists(snaplookup[sortage[j]]+'.pkl'):
    this_snap = pd.read_pickle(snaplookup[sortage[j]]+'.pkl')
else:
    s = pb.load(snaplookup[sortage[j]])
    h=s.halos()
    try:
        pb.analysis.angmom.faceon(h[1])
    except:
        print('no halo!')
        continue
    s.physical_units()
    sim = h[1]

    stars = sim.s
    if len(stars)==0:
        print('no stars!')
        continue

    try:
        stars['iord']
    except:
        print('no IDs!')
        continue
    this_snap = pd.DataFrame({k:stars[k] for k in cols})
    this_snap = pd.merge(this_snap, agepickle[['iord', 'age', 'feh', 'Rbir','zbir']], left_on='iord', right_on='iord', how='inner')
    this_snap.to_pickle(snaplookup[sortage[j]]+'.pkl')

if exists(snaplookup[sortage[j+1]]+'.pkl'):
    next_snap = pd.read_pickle(snaplookup[sortage[j+1]]+'.pkl')
else:
    s = pb.load(snaplookup[sortage[j+1]])
    h=s.halos()
    try:
        pb.analysis.angmom.faceon(h[1])
    except:
        print('no halo!')
        continue
    s.physical_units()
    sim = h[1]

    stars = sim.s
    if len(stars)==0:
        print('no stars!')
        continue

    try:
        stars['iord']
    except:
        print('no IDs!')
        continue
    next_snap = pd.DataFrame({k:stars[k] for k in cols})
    next_snap = pd.merge(next_snap, agepickle[['iord', 'age', 'feh', 'Rbir','zbir']], left_on='iord', right_on='iord', how='inner')
    next_snap.to_pickle(snaplookup[sortage[j+1]]+'.pkl')

plt.figure(figsize=(10,8))
plt.title(simsnames[i]+'; Cosmic Time = %.3f'%(allages_rs[j+1])+' Gyr', fontsize=25)

timediff = allages_rs[j+1]-allages_rs[j]

output = plt.hexbin(this_snap['age'], this_snap['Rbir'], C = this_snap['r']-this_snap['Rbir'],
       extent = (0,14,0,15), gridsize=40, reduce_C_function = np.std, mincnt = 20)
output_2 = plt.hexbin(next_snap['age'], next_snap['Rbir'], C = next_snap['r']-next_snap['Rbir'],
       extent = (0,14,0,15), gridsize=40, reduce_C_function = np.std, mincnt = 20)
diffout = (output_2.get_array - output.get_array)/timediff
plt.imshow(output.T, interpolation="nearest" ,aspect = 'auto',origin = 'lower', 
       extent = (0,14,0,15),cmap=cmap)
plt.xlabel('Age [Gyr]')
plt.ylabel('$R_{bir}$') 

cb = plt.colorbar()
cb.set_label('diff($σ_{R-R_{bir}}$)/diff(\tau) [kpc/Gyr]')

plt.tight_layout()
#plt.savefig('RM/'+simsnames[i]+'/'+str(count)+'_Rbir_diff.png')
count = count+1
#plt.close()
